In [1]:
%matplotlib
from nanorsm import *
from skimage.transform import rescale

Using matplotlib backend: Qt5Agg


In [2]:
def im_stack_rescale(im_stack,x_scale):
    
    for i in range(len(x_scale)):
        tmp = rescale(im_stack[i,:,:],(1,x_scale[i]))
        im_row, im_col = tmp.shape
        if i == 0:
        
            im_stack_scaled = np.reshape(tmp,(1,im_row,im_col))
        else:
            im_stack_num, im_stack_row,im_stack_col = np.shape(im_stack_scaled)
            row = np.maximum(im_row,im_stack_row)
            col = np.maximum(im_col,im_stack_col)
            if im_row < im_stack_row:
                r_s = int (np.round((im_stack_row-im_row)/2))
            else:
                r_s = 0
            if im_col < im_stack_col:
                c_s = int (np.round((im_stack_col-im_col)/2))
            else:
                c_s = 0
            im_stack_tmp = np.zeros((im_stack_num+1,row,col))
            im_stack_tmp[0:im_stack_num,0:im_stack_row,0:im_stack_col] = im_stack_scaled
            
            im_stack_tmp[im_stack_num,r_s:im_row+r_s,c_s:im_col+c_s] = tmp
            im_stack_scaled = im_stack_tmp
        
    return im_stack_scaled

In [35]:
#data_path = '//data//users//2023Q2//Yan_2023Q2//Fluor//spec-tomo-sample2//'
data_path = '//data//users//2020Q3//ChenWiegart_2020Q3//DataAnalysis//01_Ti_fluo_Ink4_bottom_center//fitting_with_01json_noFe//'
#sid_list = np.concatenate((np.sort(get_sid_list(['219698-219728'],3))[::-1],get_sid_list(['219620-219692'],3)))
#sid_list = get_sid_list(['222505-223042'],3)
sid_list = get_sid_list(['120248-120358'],2)

angle = np.zeros(sid_list.shape)

i = 0
for sid in sid_list:
    h = db[int(sid)]
    bl = h.table('baseline')
    angle[i] = bl['zpsth'].loc[1]
    i = i + 1

#angle = np.linspace(-90,90,181)
true_angle = angle - 2.18
true_size = np.ones(angle.shape)
ind = np.where(np.abs(angle)<45)
true_size[ind] = np.cos(true_angle[ind]*np.pi/180)/np.cos(angle[ind]*np.pi/180)*0.9542
ind2 = np.where(np.abs(angle)>=45)
true_size[ind2] = np.abs(np.sin(true_angle[ind2]*np.pi/180)/np.sin(angle[ind2]*np.pi/180))*1.0309


elem = 'Ti_K'
num = np.size(sid_list)
file_list = []
for sid in sid_list:
    tmp = ''.join([data_path, 'output_tiff_scan2D_','{}'.format(sid), '//detsum_',elem,'_norm.tiff'])
    file_list.append(tmp)
im_stack = load_ims(file_list)
im_stack = np.swapaxes(im_stack,1,2)


im_stack_rescaled = im_stack_rescale(im_stack,true_size)

'''
for i in range(len(true_size)):
    tmp = rescale(im_stack[i,:,:],(1,true_size[i]))
    im_row, im_col = tmp.shape
    if i == 0:
        
        im_stack_scaled = np.reshape(tmp,(1,im_row,im_col))
    else:
        im_stack_num, im_stack_row,im_stack_col = np.shape(im_stack_scaled)
        row = np.maximum(im_row,im_stack_row)
        col = np.maximum(im_col,im_stack_col)
        if im_row < im_stack_row:
            r_s = int (np.round((im_stack_row-im_row)/2))
        else:
            r_s = 0
        if im_col < im_stack_col:
            c_s = int (np.round((im_stack_col-im_col)/2))
        else:
            c_s = 0
        im_stack_tmp = np.zeros((im_stack_num+1,row,col))
        im_stack_tmp[0:im_stack_num,0:im_stack_row,0:im_stack_col] = im_stack_scaled
            
        im_stack_tmp[im_stack_num,r_s:im_row+r_s,c_s:im_col+c_s] = tmp
        im_stack_scaled = im_stack_tmp

'''        
num_frame,im_row,im_col = np.shape(im_stack)

im_stack_aligned, trans_matrix = align_im_stack(im_stack_rescaled) # use pystackreg
#im_stack = np.delete(im_stack,0,axis=0)
#trans_matrix = np.delete(trans_matrix,0,axis=0)

#im_stack_test = interp_sub_pix(im_stack,trans_matrix) # verify the alignment is done correctly

tifffile.imwrite(''.join([data_path, 'stack_rescaled_aligned_', elem, '.tif']),
                 im_stack_aligned.astype(np.float32),imagej=True)

Progress:   0%|          | 0/56 [00:00<?, ?it/s]

/opt/conda_envs/nsls2-analysis-2021-1.2/lib/python3.7/site-packages/pystackreg/pystackreg.py:383: UserWarning: Detected axis 1 as the possible time axis for the stack due to its low variability, but axis 0 was supplied for registration. Are you sure you supplied the correct axis?
  lowest_var_axis, axis


In [34]:
angle

array([-70., -69., -68., -67., -66., -65., -64., -63., -62., -61., -60.,
       -59., -58., -57., -56., -55., -54., -53., -52., -51., -50., -49.,
       -48., -47., -46., -45., -44., -43., -42., -41., -40., -39., -38.,
       -37., -36., -35., -34., -33., -32., -31., -30., -29., -28., -27.,
       -26., -25., -24., -23., -22., -21., -20., -19., -18., -17., -16.,
       -15.])

In [20]:
im_stack_scaled.shape

(181, 100, 109)

In [14]:
plt.figure()
plt.imshow(rescale(im_stack_aligned[0,:,:],(1,1.5)))

In [61]:
# if there are more elements to be aligned and stacked
elem_list = ['Mn_K','Fe_K','Mo_L']
#sid_list = get_sid_list(['219792-219888'],3)

sum_int = np.sum(np.sum(im_stack_test[:,10:-10,10:-10],-1),-1)
norm_factor = np.reshape(sum_int[0]/sum_int,(-1,1,1))

num = np.size(sid_list)
i = 1
for elem in elem_list:
    file_list = []
    for sid in sid_list:
        tmp = ''.join([data_path, 'output_tiff_scan2D_','{}'.format(sid), '//detsum_',elem,'_norm.tiff'])
        
        file_list.append(tmp)
    im_stack = load_ims(file_list)
    #im_stack_scaled = im_stack_rescale(im_stack,true_size)
    
    im_stack_test = interp_sub_pix(im_stack,trans_matrix)
    sz = im_stack_test.shape
    #im_stack_test = im_stack_test/np.tile(norm_factor,[1,sz[1],sz[2]])
    imp = np.sum(im_stack_test,0)
    imp.shape
    if i == 1:
        stack = imp
        sz = np.shape(imp)
        stack = np.reshape(stack, (1,sz[0],sz[1]))
    else:
        stack = np.concatenate((stack,imp[np.newaxis,:,:]),0)
    i = i+1
    tifffile.imwrite(''.join([data_path, 'stack_aligned_', elem, '_plan_52.tif']),im_stack_test[:,10:-10,0:-10].astype(np.float32),imagej=True)

Progress:   0%|          | 0/180 [00:00<?, ?it/s]

Progress:   0%|          | 0/180 [00:00<?, ?it/s]

Progress:   0%|          | 0/180 [00:00<?, ?it/s]

Progress:   0%|          | 0/180 [00:00<?, ?it/s]

Progress:   0%|          | 0/180 [00:00<?, ?it/s]

Progress:   0%|          | 0/180 [00:00<?, ?it/s]

In [44]:
plt.figure()
plt.imshow(im_stack_aligned[:,50,:])